In [1]:
import os
import tensorflow as tf

In [2]:
file_name = "int_HanDB_Test.tfrecord"
record_iterator = tf.python_io.tf_record_iterator(path=file_name)

numb_of_data=0
for record in record_iterator:
    numb_of_data+=1

tf_record_filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once("./"+file_name))
#f=open("./test_record.tfrecord",'rb')

In [3]:
def read_and_decode(filename_queue, batch_size):
    tf_record_reader=tf.TFRecordReader()
    _, tf_record_serialized = tf_record_reader.read(tf_record_filename_queue)

    tf_record_features = tf.parse_single_example(
        tf_record_serialized,
        features={
            'label': tf.FixedLenFeature([], tf.int64),
            'width': tf.FixedLenFeature([], tf.int64),
            'height': tf.FixedLenFeature([], tf.int64),
            'image': tf.FixedLenFeature([], tf.string),
        })
    tf_record_image = tf.decode_raw(tf_record_features['image'],tf.uint8)
    
    tf_record_image = tf.cast(tf_record_image, tf.float32)

    tf_record_image = tf.reshape(tf_record_image, [64,64,1])

    tf_record_label = tf.cast(tf_record_features['label'],tf.int32)
    
    images, labels = tf.train.shuffle_batch([tf_record_image, tf_record_label],
                                         batch_size = batch_size,
                                         capacity=20*batch_size,
                                         min_after_dequeue=5*batch_size )
    return images, labels

In [4]:
batch_size=50;

total_batch = int(numb_of_data / batch_size)
images, labels = read_and_decode(tf_record_filename_queue,batch_size)
print(images, labels)

Tensor("shuffle_batch:0", shape=(50, 64, 64, 1), dtype=float32) Tensor("shuffle_batch:1", shape=(50,), dtype=int32)


In [5]:
input_images = images
input_labels = labels
keep_prob = tf.placeholder(tf.float32)
print(input_images)
print(input_labels)

Tensor("shuffle_batch:0", shape=(50, 64, 64, 1), dtype=float32)
Tensor("shuffle_batch:1", shape=(50,), dtype=int32)


In [6]:
W1 = tf.Variable(tf.random_normal([5,5,1,32], stddev=0.01))
L1 = tf.nn.conv2d(input_images,W1, strides=[1,1,1,1], padding = 'SAME')
print(L1)
L1 = tf.nn.relu(L1)

L1 =  tf.nn.max_pool(L1, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L1)

Tensor("Conv2D:0", shape=(50, 64, 64, 32), dtype=float32)
Tensor("MaxPool:0", shape=(50, 32, 32, 32), dtype=float32)


In [7]:
W2 = tf.Variable(tf.random_normal([5, 5, 32, 64], stddev=0.01))

L2 = tf.nn.conv2d(L1,W2, strides=[1,1,1,1], padding = 'SAME')
L2 = tf.nn.relu(L2)

L2 =  tf.nn.max_pool(L2, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L2)

Tensor("MaxPool_1:0", shape=(50, 16, 16, 64), dtype=float32)


In [8]:
W3 = tf.Variable(tf.random_normal([4, 4, 64, 128], stddev=0.01))

L3 = tf.nn.conv2d(L2,W3, strides=[1,1,1,1], padding = 'SAME')
L3 = tf.nn.relu(L3)

L3 =  tf.nn.max_pool(L3, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L3)

Tensor("MaxPool_2:0", shape=(50, 8, 8, 128), dtype=float32)


In [9]:
W4 = tf.Variable(tf.random_normal([4, 4, 128, 256], stddev=0.01))

L4 = tf.nn.conv2d(L3,W4, strides=[1,1,1,1], padding = 'SAME')
L4 = tf.nn.relu(L4)

L4 =  tf.nn.max_pool(L4, ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
print(L4)

Tensor("MaxPool_3:0", shape=(50, 4, 4, 256), dtype=float32)


In [10]:
W5 = tf.Variable(tf.random_normal([4*4*256, 1024], stddev=0.01))
L5 = tf.reshape(L4,[-1,4*4*256])
L5 = tf.matmul(L5, W5)
L5 = tf.nn.relu(L5)

L5 = tf.nn.dropout(L5,keep_prob)
print(L5)

Tensor("dropout/mul:0", shape=(50, 1024), dtype=float32)


In [11]:
W6 = tf.Variable(tf.random_normal([1024,2350],stddev=0.01))
logit = tf.matmul(L5, W6)
print(logit)

Tensor("MatMul_1:0", shape=(50, 2350), dtype=float32)


In [12]:
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logit, labels=input_labels))
optimizer = tf.train.AdamOptimizer(0.001).minimize(loss)

In [13]:
correct_prediction = tf.equal(tf.argmax(logit,1),tf.argmax(input_labels,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [14]:
init = tf.group(tf.global_variables_initializer(),tf.local_variables_initializer())

with tf.Session() as sess:
    sess.run(init)
    coord=tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    
    for eopoch in range(3):
        for i in range(total_batch):
        #for i in range(batch_size):
            sess.run([optimizer,loss],feed_dict={keep_prob:0.5})
            train_acc=sess.run(accuracy, feed_dict={keep_prob:1.0})
            print('step %d, training accuracy %g' % (i, train_acc))
    coord.request_stop()
    coord.join(threads)

INFO:tensorflow:Error reported to Coordinator: <class 'tensorflow.python.framework.errors_impl.CancelledError'>, Enqueue operation was cancelled
	 [[Node: input_producer/input_producer_EnqueueMany = QueueEnqueueManyV2[Tcomponents=[DT_STRING], timeout_ms=-1, _device="/job:localhost/replica:0/task:0/device:CPU:0"](input_producer, input_producer/RandomShuffle)]]


InvalidArgumentError: Expected dimension in the range [-1, 1), but got 1
	 [[Node: ArgMax_1 = ArgMax[T=DT_INT32, Tidx=DT_INT32, output_type=DT_INT64, _device="/job:localhost/replica:0/task:0/device:CPU:0"](shuffle_batch:1, ArgMax_1/dimension)]]

Caused by op 'ArgMax_1', defined at:
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py", line 486, in start
    self.io_loop.start()
  File "C:\Users\JIN\Anaconda3\envs\tensorflow\lib\site-packages\zm